In [4]:
# 載入訓練好的模型
import torch
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering
import requests
url = 'https://gist.githubusercontent.com/skywalker0803r/1e1d0d25999094bf8ec23dc143e02478/raw/2b751c959a1b48a14e627eaf956f9ae3df821f46/bert_train_pipeline.py'
exec(requests.get(url).text)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
qustion = 'What is the product name?'
model_path = '../models/Product_Data_SQuAD_model_product.pt'
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load(model_path))
model.eval()

nlp = pipeline('question-answering', model=model.to('cpu'), tokenizer=tokenizer)

Global seed set to 42
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should pr

In [5]:
import re

def preprocess(x):
    x = str(x)
    x = re.sub('[\u4e00-\u9fa5]', '', x) # 去除中文
    x = re.sub(r'[^\w\s]','',x) # 去除標點符號
    x = x.replace('\n', '').replace('\r', '').replace('\t', '') # 換行符號去除
    return str.strip(x) # 移除左右空白

# 寶典
df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv',index_col=0)[['string_X_train','Y_label','EXPNO','from']].dropna(axis=0)
產品集合 = set(df['Y_label'].values.tolist())
寶典 = [ preprocess(i) for i in 產品集合]
寶典 = sorted(寶典,key=len)[::-1]

# 讀取combined_excel
df = pd.read_excel('../data/combined_excel.xlsx',index_col=0)
df = df.rename(columns={'45A':'string_X_train','SPEC':'Y_label'})
df = df[['string_X_train','Y_label','EXPNO','from']].dropna(subset=['string_X_train'],axis=0)

# 預處理
df['from'] = df['from'].apply(lambda x:''.join(x.split('-')[1:])) # 格式處理
df['string_X_train'] = df['string_X_train'].apply(preprocess) # 文本處理
df['Y_label'] = df['Y_label'].apply(lambda x:str(x).split('\n')) #可能有多個產品所以 \n 來分隔

# 執行labeling
df['p'] = 'not find'
for idx in tqdm(df.index):
    for p in df.loc[idx,'Y_label'] + 寶典:
        if p in df.loc[idx,'string_X_train']:
            df.loc[idx,'p'] = p
            continue
print('have label',df[df['p'] != 'not find'].shape)
print('no label',df[df['p'] == 'not find'].shape)

# 取沒有label到的
df = df[df['p'] == 'not find'] 
df['Y_label'] = df['p']
df = df.drop(['p'],axis=1)
df.head(10)


have label (11200, 5)
no label (4073, 5)


,string_X_train,Y_label,EXPNO,from
1,17 MT PVC COPOLYMER RESIN C15 AT USD 1470 P...,not find,NaN,20210103.xlsx
3,MASS PVC RESIN B57QUANTITY 175 MT AT 1300 USDM...,not find,11,20210103.xlsx
17,TERM OF SALE CIF MUNDRA SEAPORT INDIA17 MT OF ...,not find,NaN,20210103.xlsx
20,FOR PRESENTATION OF FOLLOWING CONFORM DOCUMENT...,not find,NaN,20210103.xlsx
22,300000 BBLS 10PCT OF GASOIL 10PPMUNIT PRICE TH...,not find,61,20210103.xlsx
23,300000 BBLS 10PCT OF GASOIL 10PPMUNIT PRICE TH...,not find,61,20210103.xlsx
24,300000 BBLS 10PCT OF GASOIL 10PPMUNIT PRICE TH...,not find,61,20210103.xlsx
30,300000 BBLS 10PCT OF GASOIL 10PPMUNIT PRICE TH...,not find,61,20210103.xlsx
52,NO COMMODITY QUANTITY UNIT PRICE AM...,not find,18,20210103.xlsx
53,NO COMMODITY QUANTITY UNIT PRICE AM...,not find,18,20210103.xlsx


In [6]:
def inference(nlp,df,qustion):
    table = pd.DataFrame()
    for i in tqdm(df.index):
        string_X_train = df.loc[i,'string_X_train']
        QA_input = {'question': qustion,'context': string_X_train}
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({
            'string_X_train':string_X_train,
            'predict':predict,
            'EXPNO':df.loc[i,'EXPNO'],
            'from':df.loc[i,'from']
            },
            index=[i])
        table = table.append(row)
    return table

In [7]:
table = inference(nlp,df,qustion)

In [8]:
table

,string_X_train,predict,EXPNO,from
1,17 MT PVC COPOLYMER RESIN C15 AT USD 1470 P...,PVC COPOLYMER RESIN C15,NaN,20210103.xlsx
3,MASS PVC RESIN B57QUANTITY 175 MT AT 1300 USDM...,PVC RESIN,11,20210103.xlsx
17,TERM OF SALE CIF MUNDRA SEAPORT INDIA17 MT OF ...,PVC COPOLYMER RESINC1517,NaN,20210103.xlsx
20,FOR PRESENTATION OF FOLLOWING CONFORM DOCUMENT...,TAISAP BC283,NaN,20210103.xlsx
22,300000 BBLS 10PCT OF GASOIL 10PPMUNIT PRICE TH...,BBLS 10PCT,61,20210103.xlsx
...,...,...,...,...
20711,COMMODITY BISPHENOLAUNIT PRICE USD 2160MTQUANT...,BISPHENOLAUNIT,NaN,20200712.xlsx
20712,COMMODITY BISPHENOLAUNIT PRICE USD 2200MTQUANT...,BISPHENOLAUNIT,NaN,20200712.xlsx
20713,DICYCLOPENTADIENEDCPDLFOB ANY PORT OF TAIWAN,DICYCLOPENTADIENEDCPDLFOB,NaN,20200712.xlsx
20718,BISPHENOLA 90000000KGS USD27KGUSD243000000CO...,BISPHENOLA,NaN,20200712.xlsx
